# Prediction with LSTM model (v.03)
<p style='text-align: right;'>with selectd4.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [2]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

restart kernel... 2025-05-23 16:26:45.290853


In [1]:
# load dependacies
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Input

from globalvar import *
from data_loader import DataLoader
from model1 import PredictLSTM1
from model2 import PredictLSTM2
from model3 import PredictLSTM3
from model4 import PredictLSTM4

import argparse
import matplotlib.pyplot as plt

from util_pred import print_data
from util_pred import flat_data_with_sum, flat_data, get_frequency
from util_pred import save_model, import_mode
from util_pred import get_random_in_list
from util_pred import print_data_with_sort
from util_pred import get_sorted_n_values
from util_pred import dict_key_count
from util_pred import print_list
from util_pred import print_dict_list
from util_pred import print_title
from activation import ActivationOutput, RecurrentActivation
from datetime import datetime

""" db """
from db_operate import conn_db, close_db
from db_operate import init_metric
from db_operate import select_metric, delete_all_metric
from db_operate import insert_metric

from operate import create_model, create_model_v1, get_predicted, create_model_v2

print(f"imported library. ({datetime.now()})")

imported library. (2025-05-23 16:27:04.138799)


In [3]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./selectd3.csv'
    
mode='predict' # help = "back-test or predict")
mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=0.95 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
last = [[7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")
MAX_MODEL_CNT = 5

write_to_db = True


finished to set environemnt. (2025-05-23 16:27:22.181854)


In [4]:
# data prepare and set base model
dataset = DataLoader(data_dir=data_dir,
                     training_length=training_length,
                     window_prev=window,
                     mode=mode,
                     from_pos=from_pos
                    )

dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300
                  )
    }
print(f'test_X = {dataset_dicts[1].test_X}')
print(f'completed to load data. {datetime.now()}')

DataLoader.preproc_csv origin = 571, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]
 [1171    3    6 ...   11   12   17]]
rDataLoader.preproc_csv aw_np_proc = 571, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [12 24 26 39 42]
 [13 28 34 38 42]
 [ 6  7 11 12 17]]
DataLoader.preproc_csv origin = 571, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]
 [1171    3    6 ...   11   12   17]]
rDataLoader.preproc_csv aw_np_proc = 571, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [12 24 26 39 42]
 [13 28 34 38 42]
 [ 6  7 11 12 17]]
DataLoader.preproc_csv origin = 571, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [116

In [5]:
###########
## layers
## LSTM Neural 계층 선언
###########
"""
Activation (Output):
    linear: No activation, output is directly passed through.
    relu: Rectified Linear Unit, max(x, 0).
    sigmoid: Sigmoid function, output between 0 and 1.
    tanh: Hyperbolic tangent, output between -1 and 1.
    softmax: Normalizes output to a probability distribution.
    elu: Exponential Linear Unit.
    selu: Scaled Exponential Linear Unit.

Recurrent Activation:
    sigmoid: Commonly used for gates in LSTM.
    hard_sigmoid: A faster, less computationally expensive version of sigmoid.
    tanh: Can be used, but sigmoid is more typical for gates.
"""
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

lstm_units=[[(45, ActivationOutput.selu.name),   # 0
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.selu.name),   # 1
             (45, ActivationOutput.selu.name)
             ],
            [(45, ActivationOutput.selu.name),   # 2
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.tanh.name),   # 3
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name)
             ],
            [(45, ActivationOutput.tanh.name),   # 4
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             ]]
dense_units = [[(45, ActivationOutput.selu.name),   # 0
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.selu.name),], # 1
               [(45, ActivationOutput.selu.name),   # 2
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.tanh.name),   # 3
               ],
               []                                   # 4
              ]
sel_date = datetime(2025, 5, 17, 20, 35)
sel_date_ts = (sel_date.timestamp() / 10000000)
steps = [45, 45,]
metrics = [["accuracy"]]
dropout = [0, 0]
learning_rate = [0.01, 0.05]
last_lstm_return_sequences = [False]
loss = ["binary_crossentropy", "mse"]
output_dense_activation = [ActivationOutput.elu.name, #0
                           ActivationOutput.selu.name, #1
                           ActivationOutput.sigmoid.name, #2
                           ActivationOutput.tanh.name, #3
                           ActivationOutput.softmax.name, #4
                           ActivationOutput.relu.name, #5
                           ActivationOutput.linear.name #6
                          ]
epochs = [10, #0
          20, #1
          25, #2
          50, #3
          100] #4

rand_seed=[sel_date_ts, datetime.now().timestamp(), 0]


lstm_args = {1: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[0],
                          "dense_units": dense_units[0],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             2: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[1],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             3: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[1],
                          "dense_units": dense_units[1],
                          "output_dense_activation": output_dense_activation[0],
                          "loss": loss[0],
                          "rand_seed": rand_seed[1]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 1
                           },
                 "dataset": datasets[0]
                 },
             4: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[2],
                          "dense_units": dense_units[2],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "rand_seed": rand_seed[2]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 100
                           },
                 "dataset": datasets[0]
                 },
             5: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[3],
                          "dense_units": dense_units[3],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "rand_seed": rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 100
                           },
                 "dataset": datasets[0]
                 },
             }


models = [None for i in range(MAX_MODEL_CNT)]

matched_cnts = {}
selected_fives = {}
matched_list = {}
predicted_all = {}

print(f'completed to set evn for all models. {datetime.now()}')

completed to set evn for all models. 2025-05-23 16:27:34.756133


In [10]:
import json
import os
from datetime import datetime

def listinlist_2_strinlist(list_in_list):
    """ listinlist_2_strinlist """
    str_in_list = []
    for l in list_in_list:
        str_in_list.append(",".join(l))
    return str_in_list      


def change_matched_info(matched_count, matched_list):
    counts = {}
    cnt = 1
    for c in matched_count:
        counts[cnt] = ",".join([ str(i) for i in c])
        cnt += 1
    cnt = 1
    datas = {}
    for d in matched_list:
        datas[cnt] = d
        cnt += 1
    return counts, datas

def write_json(trial, matched_count, matched_list, append=False):
    suffix = datetime.now().strftime('%y%m%d_%H')
    file_name = f"matched_{suffix}.json"
    matched_dict = {}
    if append:
        if os.path.isfile(file_name):
            with open(file_name, "r") as file:
                matched_dict = json.load(file)
                file.close()
    counts, datas = change_matched_info(matched_count, matched_list)
    if len(counts) > 0 and len(datas) > 0:
        matched_dict[trial] = {}
        matched_dict[trial]["counts"] =  counts
        matched_dict[trial]["datas"] = datas
        print(f'matched_dict = {matched_dict}')
        with open(file_name, "w") as file:
            json.dump(matched_dict, file, indent=4)

In [19]:
# all (SMALL): predict #2
def predict_small(trial, last, models, verbose=True):
    matched_cnts_small = [0 for i in range(MAX_MODEL_CNT)]
    selected_fives_small = [None for i in range(MAX_MODEL_CNT)]
    matched_list_small = [None for i in range(MAX_MODEL_CNT)]
    predicted_all_small = [None for i in range(MAX_MODEL_CNT)]

    for i in range(MAX_MODEL_CNT):
        title = f"history #{i+1}"
        model_num = i
        if models[model_num] is None:
            print(f"model #{model_num+1} is None")
        else:
            models[model_num][0].verb = "None"
            matched_cnts_small[model_num], \
            selected_fives_small[model_num], \
            matched_list_small[model_num], \
            predicted_all_small[model_num] = get_predicted(
                title=title,
                model=models[model_num][0],
                mode=mode2,
                use_pre=False,
                last=last,
                verbose=verbose,
                trial=5
            )
    print(f'predicted time : trial={trial} date={datetime.now()}')
    return matched_cnts_small, matched_list_small


# all (SMALL): print predicted
def print_predict_small(matched_cnts_small, matched_list_small):
    print(f'print time : {datetime.now()}')
    #write_json(trial = 3, matched_count=matched_cnts_small, matched_list=matched_list_small, append=True)
    print_dict_list(title="matched_cnt", datas=matched_cnts_small)
    for i in range(MAX_MODEL_CNT):
        print_list(title=f"matched_list[{i+1}]", datas=matched_list_small[i])

In [102]:
single_run = False
if single_run:
    model = create_model_v2(id=1,
                            model_type="lstm4",
                            lstm_info=lstm_args[1],
                            dataset=datasets[1],
                            verbose=True)

In [7]:
print(f'current_active_version={model_version}')

current_active_version=10


In [32]:
# all : create model and training
model_version = 0
with open('active.version', 'r') as fd:
    line = fd.read()
    model_version = int(line)
print(f'previous model_version={model_version}')
print(f'open model version file {datetime.now()}')

print(f'start to train all models. {datetime.now()}')
for i in range(MAX_MODEL_CNT):
    print(f'{i+1}\'s training. status=start {datetime.now()}')
    model = create_model_v2(id=i,
                         model_type="lstm4",
                         lstm_info=lstm_args[i+1],
                         dataset=lstm_args[i+1]["dataset"],
                         verbose=False
                        )
    models[i] = (model, lstm_args[i+1])
    print(f'{i+1}\'s training. status=end {datetime.now()}')
print(f'completed to train all models. {datetime.now()}')
model_version += 1
print(f'check to create model({model_version}). {datetime.now()}')
current = 0
data_cols = []
with open('active.version', 'w') as fd:
    fd.write(f'{model_version}')

previous model_version=17
open model version file 2025-05-23 17:55:27.369285
start to train all models. 2025-05-23 17:55:27.369375
1's training. status=start 2025-05-23 17:55:27.369587
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
lstm unit : 3, (45, 'sigmoid')
lstm unit : 4, (45, 'elu')
loss = binary_crossentropy
verbose=0
model=<Sequential name=sequential_35, built=True>
num_epoch=50/<class 'int'>
1's training. status=end 2025-05-23 17:56:25.320141
2's training. status=start 2025-05-23 17:56:25.320432
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
loss = binary_crossentropy
verbose=0
model=<Sequential name=sequential_36, built=True>
num_epoch=20/<class 'int'>
2's training. status=end 2025-05-23 17:56:45.678757
3's training. status=start 2025-05-23 17:56:45.679052
lstm unit : 1, (45, 'selu')
lstm unit : 2, (45, 'selu')
loss = binary_crossentropy
verbose=0
model=<Sequential name=sequential_37, built=True>
num_epoch=50/<class 'int'>
3's training. status=end 2025-05-23

In [33]:
print(f'check to create model({model_version}). {datetime.now()}')

check to create model(18). 2025-05-23 17:59:09.566067


In [106]:
view_model_info = False
if view_model_info:
    for i in range(MAX_MODEL_CNT):
        print(models[i][0].model.summary())

In [108]:
# all with trial : print predicted
run = False
if run:
    write_json(trial = 3, matched_count=matched_cnts, matched_list=matched_list, append=True)
    print_dict_list(title="matched_cnt", datas=matched_cnts)
    for i in range(MAX_MODEL_CNT):
        print_list(title=f"matched_list[{i+1}]", datas=matched_list[i])

In [34]:
if write_to_db:
    conn = conn_db('metrics.db')
    need_db_init = False
    init_metric(conn, drop_table=need_db_init)
else:
    conn = None
round_limit=40
for round in range(round_limit+1):
    matched_cnts_all = []
    matched_list = []
    date = str(datetime.now())
    matched_cnts_small, matched_list_small = predict_small(trial=round, 
                                                           last=last,
                                                           models=models,
                                                           verbose=False
                                                          )
    #print_predict_small(matched_cnts_small, matched_list_small)
    matched_cnts_all.append(matched_cnts_small)
    matched_list.append(matched_list_small)
    insert_metric(conn=conn, version=model_version, round=round, date=date, model_datas=matched_list)
if write_to_db:
    close_db(conn)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step
predicted time : trial=0 date=2025-05-23 17:59:20.018637
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
predicted time : trial=1 date=2025-05-23 17:59:20.620395
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
predicted time : trial=2 date=2025-05-23 17:59:21.204394
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
predicted time : trial=3 date=20

In [ ]:
trial = 1
matched_cnts_all = []
matched_list = []
for t in range(1, trial+1):
    matched_cnts_small, matched_list_small = predict_small(trial=t, 
                                                           last=last,
                                                           models=models)
    #print_predict_small(matched_cnts_small, matched_list_small)
    matched_cnts_all.append(matched_cnts_small)
    matched_list.append(matched_list_small)
current +=1


In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()

In [ ]:
# all : model info
#################
# model info
#################
for model in models:
    if model is not None:
        print(f'model:\n\t{model[0].model.summary()}')
        print(f'layer:\n\t{model[1]}')

In [ ]:
# all : model test
mode = "back-test"
if mode == 'back-test':
    for model in models:
        if model is not None:
            greed_prediction_number_set = model[0].predict_numbers("greed", trial=1)
            random_pred_set = model[0].predict_randomely(trial=1)
            model[0].evaluate(greed_prediction_number_set)
            print("---------Random baseline-------------")
            model[0].evaluate(random_pred_set)

In [ ]:
for i in range(3):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
# one : create model and training
id = 6
print(f'start to train. [id={id}]')
models[6] = create_model_v1(id=6, dataset=dataset, epoch=10, verbose=True)
print(f'model = {models[6]}')
print(f'end to train. [id={id}]')

In [ ]:
# [3, 6, 7, 11, 12, 17]

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set2, 5)}')

In [ ]:
print_data(title='greed_predicted',
           data_set=greed_prediction_number_set,
           add_val=0, need_sort=True)
print(f'greed_prediction_number_set\n\t\t{greed_prediction_number_set}')

In [ ]:
# when 
"""
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=100 # default = 3
batch=3 # default = 1
model='lstm2' # help = "lstm or lstm2")
hid_dim = 128
"""
for i in range(6):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
print(f'model={model}')

In [ ]:
random_pred_set = model2.predict_randomely(trial=1)
print(f'prediction_number_set = {random_pred_set}')

In [ ]:
# 1170 : 3·13·28·34·38·42

last = [
    [6, 8, 37, 40, 41, 44],
    [8, 10, 20, 25, 33, 37],
    [4, 18, 27, 32, 40, 43],
    [2, 8, 10, 31, 33, 35],
    [15, 20, 28, 38, 41, 45],
    [12, 26, 13, 5, 31, 38]
]

In [ ]:
#1171 : [3, 6, 7, 11, 12, 17]

```
sqlite> select * from sqlite_master;
table|metrics|metrics|2|CREATE TABLE metrics (
            id INTEGER PRIMARY KEY,
            version INTEGER,
            round INTEGER,
            model_id INTEGER, 
            trial INTEGER,
            date DATETIME,
            metric TEXT,
            matched TEXT,
            matched_size INTEGER
        )
```

qlite> select * from metrics where version=18 and model_id=5 and matched_size >= 3;
27847|18|6|5|2|2025-05-23 17:59:23.004193|9,44,34,7,16,21|7,9,44|3
27849|18|6|5|4|2025-05-23 17:59:23.004193|7,24,29,33,37,42|7,24,42|3
27950|18|10|5|5|2025-05-23 17:59:25.393964|1,7,17,24,38,42|7,24,42|3
27974|18|11|5|4|2025-05-23 17:59:26.000667|1,7,26,32,40,44|7,40,44|3
28424|18|29|5|4|2025-05-23 17:59:38.549303|1,9,11,24,29,42|9,24,42|3
28646|18|38|5|1|2025-05-23 17:59:44.194185|1,44,24,35,36,42|24,42,44|3
sqlite> select * from metrics where version=17 and model_id=5 and matched_size >= 3;
26847|17|7|5|2|2025-05-23 17:49:23.152138|9,23,24,42,43,45|9,24,42|3
sqlite> select * from metrics where version=16 and model_id=5 and matched_size >= 3;
25822|16|7|5|2|2025-05-23 17:41:36.814124|14,28,34,40,42,44|40,42,44|3
26023|16|15|5|3|2025-05-23 17:41:41.590253|5,7,9,15,21,44|7,9,44|3
26025|16|15|5|5|2025-05-23 17:41:41.590253|37,43,9,24,39,44|9,24,44|3
26196|16|22|5|1|2025-05-23 17:41:45.781793|44,24,34,42,32,25|24,42,44|3
26224|16|23|5|4|2025-05-23 17:41:46.370263|20,24,29,30,40,44|24,40,44|3
26371|16|29|5|1|2025-05-23 17:41:49.986134|44,40,3,42,6,22|40,42,44|3
sqlite> select * from metrics where version=15 and model_id=5 and matched_size >= 3;
24696|15|3|5|1|2025-05-23 17:33:15.720394|5,7,9,10,12,44|7,9,44|3
25248|15|25|5|3|2025-05-23 17:33:29.056313|7,9,33,39,40,45|7,9,40|3
25275|15|26|5|5|2025-05-23 17:33:29.660071|24,8,7,9,25,43|7,9,24|3
25450|15|33|5|5|2025-05-23 17:33:33.983810|18,20,25,40,42,44|40,42,44|3
sqlite> select * from metrics where version=14 and model_id=5 and matched_size >= 3;
23675|14|3|5|5|2025-05-23 17:07:31.378363|8,17,27,40,42,44|40,42,44|3
23796|14|8|5|1|2025-05-23 17:07:34.342720|9,38,24,42,15,41|9,24,42|3
23824|14|9|5|4|2025-05-23 17:07:34.945395|2,7,8,16,24,44|7,24,44|3
24025|14|17|5|5|2025-05-23 17:07:39.736216|6,9,26,29,42,44|9,42,44|3
24522|14|37|5|2|2025-05-23 17:07:52.591513|7,11,14,32,40,42|7,40,42|3
sqlite> select * from metrics where version=13 and model_id=5 and matched_size >= 3;
23099|13|21|5|4|2025-05-23 16:58:38.093198|26,3,24,40,7,38|7,24,40|3
23175|13|24|5|5|2025-05-23 16:58:39.908675|44,4,42,12,9,29|9,42,44|3
23499|13|37|5|4|2025-05-23 16:58:47.795038|12,34,39,40,42,44|40,42,44|3
23573|13|40|5|3|2025-05-23 16:58:49.983623|2,24,26,33,42,44|24,42,44|3
sqlite> select * from metrics where version=12 and model_id=5 and matched_size >= 3;
21723|12|7|5|3|2025-05-23 16:47:30.740589|24,7,1,4,9,6|7,9,24|3
21924|12|15|5|4|2025-05-23 16:47:36.077104|6,7,9,14,24,42|7,9,24,42|4
22500|12|38|5|5|2025-05-23 16:47:51.353032|1,9,26,41,42,44|9,42,44|3
sqlite> select * from metrics where version=11 and model_id=5 and matched_size >= 3;
20524|11|0|5|4|2025-05-23 16:32:39.629680|7,9,11,16,32,40|7,9,40|3
20673|11|6|5|3|2025-05-23 16:32:48.725120|7,14,24,39,40,43|7,24,40|3
20723|11|8|5|3|2025-05-23 16:32:49.895927|37,45,24,40,3,7|7,24,40|3
20847|11|13|5|2|2025-05-23 16:32:54.577514|7,9,10,14,30,44|7,9,44|3
20849|11|13|5|4|2025-05-23 16:32:54.577514|4,9,20,28,40,42|9,40,42|3
20925|11|16|5|5|2025-05-23 16:32:56.411649|7,9,44,36,24,39|7,9,24,44|4
sqlite> select * from metrics where version=10 and model_id=5 and matched_size >= 3;
19547|10|2|5|2|2025-05-23 00:21:15.773460|9,17,25,28,42,44|9,42,44|3
19550|10|2|5|5|2025-05-23 00:21:15.773460|3,9,24,31,34,44|9,24,44|3
19673|10|7|5|3|2025-05-23 00:21:17.834714|9,17,20,24,31,44|9,24,44|3
20346|10|34|5|1|2025-05-23 00:21:29.087991|2,5,7,9,24,39|7,9,24|3
20373|10|35|5|3|2025-05-23 00:21:29.493544|42,4,40,35,37,7|7,40,42|3
sqlite> select * from metrics where version=9 and model_id=5 and matched_size >= 3;
19072|9|24|5|2|2025-05-23 00:16:27.326848|9,17,38,40,44,45|9,40,44|3
19074|9|24|5|4|2025-05-23 00:16:27.326848|23,35,42,9,32,24|9,24,42|3
19349|9|35|5|4|2025-05-23 00:16:32.834460|2,7,10,24,12,42|7,24,42|3
19374|9|36|5|4|2025-05-23 00:16:33.324248|42,7,9,19,15,36|7,9,42|3
sqlite> select * from metrics where version=8 and model_id=5 and matched_size >= 3;
17649|8|8|5|4|2025-05-23 00:01:37.008797|2,7,18,24,32,40|7,24,40|3
17897|8|18|5|2|2025-05-23 00:01:41.011650|1,7,9,10,40,43|7,9,40|3
18047|8|24|5|2|2025-05-23 00:01:43.405982|40,15,34,42,20,44|40,42,44|3
18149|8|28|5|4|2025-05-23 00:01:44.942742|21,15,7,44,29,24|7,24,44|3
18198|8|30|5|3|2025-05-23 00:01:45.779000|4,7,24,36,39,44|7,24,44|3
18225|8|31|5|5|2025-05-23 00:01:46.191114|7,24,26,38,40,45|7,24,40|3
18272|8|33|5|2|2025-05-23 00:01:46.980430|7,9,19,32,33,42|7,9,42|3
18322|8|35|5|2|2025-05-23 00:01:47.758777|9,34,37,38,42,44|9,42,44|3
sqlite> select * from metrics where version=7 and model_id=5 and matched_size >= 3;
16648|7|9|5|3|2025-05-22 23:55:40.887280|6,16,24,29,40,44|24,40,44|3
16773|7|14|5|3|2025-05-22 23:55:42.889254|13,15,24,28,40,42|24,40,42|3
16997|7|23|5|2|2025-05-22 23:55:46.422761|42,24,17,40,3,36|24,40,42|3
17097|7|27|5|2|2025-05-22 23:55:47.998755|1,16,24,27,42,44|24,42,44|3
17196|7|31|5|1|2025-05-22 23:55:49.657194|21,24,29,39,40,42|24,40,42|3
17250|7|33|5|5|2025-05-22 23:55:50.503705|40,22,9,25,13,7|7,9,40|3
17423|7|40|5|3|2025-05-22 23:55:53.300663|7,10,44,19,40,30|7,40,44|3

sqlite> select * from metrics where version=18 and model_id=4 and matched_size >= 3;
27691|18|0|4|1|2025-05-23 17:59:13.781050|7,42,38,40,35,23|7,40,42|3
28020|18|13|4|5|2025-05-23 17:59:27.205381|7,9,23,25,27,44|7,9,44|3
28117|18|17|4|2|2025-05-23 17:59:29.611085|4,9,21,24,27,42|9,24,42|3
28494|18|32|4|4|2025-05-23 17:59:40.422316|7,34,37,39,40,42|7,40,42|3
28520|18|33|4|5|2025-05-23 17:59:41.065684|7,9,11,21,31,42|7,9,42|3
28594|18|36|4|4|2025-05-23 17:59:42.941789|11,17,24,35,40,44|24,40,44|3
28619|18|37|4|4|2025-05-23 17:59:43.548926|18,7,43,44,32,9|7,9,44|3
sqlite> select * from metrics where version=17 and model_id=4 and matched_size >= 3;
26966|17|12|4|1|2025-05-23 17:49:26.135680|1,7,15,31,40,42|7,40,42|3
27095|17|17|4|5|2025-05-23 17:49:29.135933|9,24,28,7,3,43|7,9,24|3
27341|17|27|4|1|2025-05-23 17:49:35.175580|44,35,40,8,25,24|24,40,44|3
27569|17|36|4|4|2025-05-23 17:49:40.702149|5,7,22,32,40,44|7,40,44|3
sqlite> select * from metrics where version=16 and model_id=4 and matched_size >= 3;
25792|16|6|4|2|2025-05-23 17:41:36.217746|4,9,24,31,41,44|9,24,44|3
25818|16|7|4|3|2025-05-23 17:41:36.814124|14,24,26,31,40,42|24,40,42|3
25966|16|13|4|1|2025-05-23 17:41:40.381148|21,9,24,7,2,14|7,9,24|3
26120|16|19|4|5|2025-05-23 17:41:43.968679|7,9,25,28,42,44|7,9,42,44|4
26394|16|30|4|4|2025-05-23 17:41:50.577066|7,17,24,25,37,40|7,24,40|3
sqlite> select * from metrics where version=15 and model_id=4 and matched_size >= 3;
25466|15|34|4|1|2025-05-23 17:33:34.583142|6,7,9,15,35,40|7,9,40|3
25591|15|39|4|1|2025-05-23 17:33:37.623474|2,7,17,20,42,44|7,42,44|3
sqlite> select * from metrics where version=14 and model_id=4 and matched_size >= 3;
23816|14|9|4|1|2025-05-23 17:07:34.945395|3,13,22,24,40,44|24,40,44|3
24043|14|18|4|3|2025-05-23 17:07:41.173053|7,21,20,40,38,24|7,24,40|3
24116|14|21|4|1|2025-05-23 17:07:42.967531|1,9,36,37,40,42|9,40,42|3
24569|14|39|4|4|2025-05-23 17:07:53.796177|4,7,21,24,41,42|7,24,42|3
sqlite> select * from metrics where version=13 and model_id=4 and matched_size >= 3;
22695|13|5|4|5|2025-05-23 16:58:28.447762|7,9,19,44,30,37|7,9,44|3
22720|13|6|4|5|2025-05-23 16:58:29.047154|7,13,20,22,40,44|7,40,44|3
23244|13|27|4|4|2025-05-23 16:58:41.751352|1,6,9,20,24,42|9,24,42|3
23544|13|39|4|4|2025-05-23 16:58:49.375029|9,35,44,25,7,32|7,9,44|3
sqlite> select * from metrics where version=12 and model_id=4 and matched_size >= 3;
22242|12|28|4|2|2025-05-23 16:47:44.710109|9,15,20,24,42,45|9,24,42|3
sqlite> select * from metrics where version=11 and model_id=4 and matched_size >= 3;
20642|11|5|4|2|2025-05-23 16:32:48.137249|20,21,24,40,42,45|24,40,42|3
20667|11|6|4|2|2025-05-23 16:32:48.725120|7,9,10,20,28,44|7,9,44|3
20716|11|8|4|1|2025-05-23 16:32:49.895927|29,34,37,40,24,42|24,40,42|3
20745|11|9|4|5|2025-05-23 16:32:50.490078|42,27,38,9,28,7|7,9,42|3
21017|11|20|4|2|2025-05-23 16:32:58.846071|42,44,7,16,14,28|7,42,44|3
21443|11|37|4|3|2025-05-23 16:33:10.221746|9,44,6,16,7,35|7,9,44|3
21467|11|38|4|2|2025-05-23 16:33:10.898083|7,19,30,40,42,44|7,40,42,44|4
sqlite> select * from metrics where version=10 and model_id=4 and matched_size >= 3;
19893|10|16|4|3|2025-05-23 00:21:21.703202|5,7,28,41,42,44|7,42,44|3
20043|10|22|4|3|2025-05-23 00:21:24.120759|9,25,30,39,40,42|9,40,42|3
20241|10|30|4|1|2025-05-23 00:21:27.402840|9,22,24,28,32,44|9,24,44|3
20267|10|31|4|2|2025-05-23 00:21:27.854761|18,36,9,7,42,6|7,9,42|3
20393|10|36|4|3|2025-05-23 00:21:29.883720|7,9,10,15,22,40|7,9,40|3
20493|10|40|4|3|2025-05-23 00:21:31.756315|9,24,26,29,38,40|9,24,40|3
sqlite> select * from metrics where version=9 and model_id=4 and matched_size >= 3;
18470|9|0|4|5|2025-05-23 00:15:56.632504|15,42,38,32,7,40|7,40,42|3
18642|9|7|4|2|2025-05-23 00:16:18.898841|4,7,24,36,40,41|7,24,40|3
18691|9|9|4|1|2025-05-23 00:16:19.868660|2,7,20,23,42,44|7,42,44|3
18695|9|9|4|5|2025-05-23 00:16:19.868660|9,40,34,22,44,18|9,40,44|3
19318|9|34|4|3|2025-05-23 00:16:32.292792|23,24,9,38,40,11|9,24,40|3
sqlite> select * from metrics where version=8 and model_id=4 and matched_size >= 3;
17494|8|2|4|4|2025-05-23 00:01:34.613208|5,7,9,15,19,42|7,9,42|3
17795|8|14|4|5|2025-05-23 00:01:39.378798|13,17,40,42,43,44|40,42,44|3
17842|8|16|4|2|2025-05-23 00:01:40.193111|7,11,18,28,40,44|7,40,44|3
17916|8|19|4|1|2025-05-23 00:01:41.396609|33,24,37,9,44,19|9,24,44|3
17941|8|20|4|1|2025-05-23 00:01:41.848351|6,7,9,16,17,24|7,9,24|3
18042|8|24|4|2|2025-05-23 00:01:43.405982|2,9,19,31,40,44|9,40,44|3
18091|8|26|4|1|2025-05-23 00:01:44.181269|8,9,10,13,40,42|9,40,42|3
18442|8|40|4|2|2025-05-23 00:01:49.696193|1,3,7,24,31,44|7,24,44|3
18444|8|40|4|4|2025-05-23 00:01:49.696193|30,23,40,42,9,19|9,40,42|3
sqlite> select * from metrics where version=7 and model_id=4 and matched_size >= 3;
16569|7|6|4|4|2025-05-22 23:55:39.586747|40,7,42,44,3,28|7,40,42,44|4
16570|7|6|4|5|2025-05-22 23:55:39.586747|1,9,24,27,39,44|9,24,44|3
16745|7|13|4|5|2025-05-22 23:55:42.476674|13,28,33,9,40,44|9,40,44|3
16994|7|23|4|4|2025-05-22 23:55:46.422761|28,44,8,40,25,9|9,40,44|3
17244|7|33|4|4|2025-05-22 23:55:50.503705|6,22,23,40,42,44|40,42,44|3
17344|7|37|4|4|2025-05-22 23:55:52.077547|7,10,19,24,27,44|7,24,44|3

sqlite> select * from metrics where version=18 and model_id=3 and matched_size >= 3;
27690|18|0|3|5|2025-05-23 17:59:13.781050|9,4,42,7,31,18|7,9,42|3
27740|18|2|3|5|2025-05-23 17:59:20.630370|9,14,33,39,40,42|9,40,42|3
28040|18|14|3|5|2025-05-23 17:59:27.816821|9,17,22,24,26,40|9,24,40|3
28061|18|15|3|1|2025-05-23 17:59:28.411285|9,13,15,24,40,44|9,24,40,44|4
28115|18|17|3|5|2025-05-23 17:59:29.611085|5,6,7,9,26,40|7,9,40|3
28489|18|32|3|4|2025-05-23 17:59:40.422316|40,26,9,2,42,12|9,40,42|3
sqlite> select * from metrics where version=17 and model_id=3 and matched_size >= 3;
26714|17|2|3|4|2025-05-23 17:49:19.878523|13,7,44,21,42,25|7,42,44|3
27287|17|25|3|2|2025-05-23 17:49:33.979821|44,38,27,8,9,24|9,24,44|3
27438|17|31|3|3|2025-05-23 17:49:37.633339|1,10,23,24,40,44|24,40,44|3
sqlite> select * from metrics where version=16 and model_id=3 and matched_size >= 3;
25636|16|0|3|1|2025-05-23 17:41:27.067406|1,5,7,9,30,40|7,9,40|3
25665|16|1|3|5|2025-05-23 17:41:33.243232|5,7,9,32,38,40|7,9,40|3
25686|16|2|3|1|2025-05-23 17:41:33.849740|1,9,23,24,30,42|9,24,42|3
25713|16|3|3|3|2025-05-23 17:41:34.447735|3,7,9,21,36,40|7,9,40|3
25838|16|8|3|3|2025-05-23 17:41:37.410198|7,9,17,20,38,44|7,9,44|3
26137|16|20|3|2|2025-05-23 17:41:44.572431|1,7,10,14,40,44|7,40,44|3
26237|16|24|3|2|2025-05-23 17:41:46.972772|42,17,24,40,14,19|24,40,42|3
26313|16|27|3|3|2025-05-23 17:41:48.775028|5,16,20,40,42,44|40,42,44|3
26461|16|33|3|1|2025-05-23 17:41:52.395268|7,8,21,24,38,42|7,24,42|3
26465|16|33|3|5|2025-05-23 17:41:52.395268|15,24,26,36,40,44|24,40,44|3
26589|16|38|3|4|2025-05-23 17:41:55.439112|9,22,40,41,42,44|9,40,42,44|4
sqlite> select * from metrics where version=15 and model_id=3 and matched_size >= 3;
24686|15|3|3|1|2025-05-23 17:33:15.720394|40,32,30,23,24,7|7,24,40|3
24715|15|4|3|5|2025-05-23 17:33:16.308455|24,40,36,9,45,41|9,24,40|3
24988|15|15|3|3|2025-05-23 17:33:22.931798|14,16,24,40,41,44|24,40,44|3
25290|15|27|3|5|2025-05-23 17:33:30.255667|7,9,24,28,34,42|7,9,24,42|4
25340|15|29|3|5|2025-05-23 17:33:31.501705|9,11,12,22,24,40|9,24,40|3
25415|15|32|3|5|2025-05-23 17:33:33.341491|1,24,29,31,40,44|24,40,44|3
25486|15|35|3|1|2025-05-23 17:33:35.175485|4,7,9,15,24,44|7,9,24,44|4
25490|15|35|3|5|2025-05-23 17:33:35.175485|44,19,25,24,21,9|9,24,44|3
sqlite> select * from metrics where version=14 and model_id=3 and matched_size >= 3;
23688|14|4|3|3|2025-05-23 17:07:31.970930|9,20,28,29,40,44|9,40,44|3
23739|14|6|3|4|2025-05-23 17:07:33.144533|7,9,30,34,35,44|7,9,44|3
23913|14|13|3|3|2025-05-23 17:07:37.319032|4,9,20,40,41,42|9,40,42|3
23964|14|15|3|4|2025-05-23 17:07:38.521982|7,14,18,23,40,42|7,40,42|3
24262|14|27|3|2|2025-05-23 17:07:46.558165|33,17,44,37,40,42|40,42,44|3
24313|14|29|3|3|2025-05-23 17:07:47.766466|2,7,13,37,40,44|7,40,44|3
24436|14|34|3|1|2025-05-23 17:07:50.793204|9,40,44,17,19,2|9,40,44|3
sqlite> select * from metrics where version=13 and model_id=3 and matched_size >= 3;
22561|13|0|3|1|2025-05-23 16:58:19.972524|18,21,24,40,43,44|24,40,44|3
23388|13|33|3|3|2025-05-23 16:58:45.380911|2,9,10,40,43,44|9,40,44|3
23465|13|36|3|5|2025-05-23 16:58:47.180013|10,24,28,33,40,44|24,40,44|3
23538|13|39|3|3|2025-05-23 16:58:49.375029|24,28,19,42,34,40|24,40,42|3
sqlite> select * from metrics where version=12 and model_id=3 and matched_size >= 3;
21687|12|6|3|2|2025-05-23 16:47:30.078557|7,10,24,29,42,44|7,24,42,44|4
22012|12|19|3|2|2025-05-23 16:47:38.737581|7,25,36,42,44,45|7,42,44|3
22240|12|28|3|5|2025-05-23 16:47:44.710109|44,40,2,35,7,25|7,40,44|3
sqlite> select * from metrics where version=11 and model_id=3 and matched_size >= 3;
20787|11|11|3|2|2025-05-23 16:32:51.660969|7,9,17,25,29,44|7,9,44|3
20965|11|18|3|5|2025-05-23 16:32:57.611828|7,14,24,41,42,43|7,24,42|3
21063|11|22|3|3|2025-05-23 16:33:00.656169|7,9,12,19,33,42|7,9,42|3
21114|11|24|3|4|2025-05-23 16:33:01.997524|24,7,26,35,40,18|7,24,40|3
21240|11|29|3|5|2025-05-23 16:33:05.209697|5,7,22,41,42,44|7,42,44|3
21340|11|33|3|5|2025-05-23 16:33:07.703573|7,36,42,26,4,9|7,9,42|3
21362|11|34|3|2|2025-05-23 16:33:08.354312|37,17,42,9,40,36|9,40,42|3
sqlite> select * from metrics where version=10 and model_id=3 and matched_size >= 3;
19486|10|0|3|1|2025-05-23 00:21:10.949664|43,3,9,14,44,7|7,9,44|3
19511|10|1|3|1|2025-05-23 00:21:15.392145|2,5,7,40,41,44|7,40,44|3
19539|10|2|3|4|2025-05-23 00:21:15.773460|9,24,29,30,40,41|9,24,40|3
19737|10|10|3|2|2025-05-23 00:21:19.168223|3,7,9,11,24,44|7,9,24,44|4
20062|10|23|3|2|2025-05-23 00:21:24.521831|7,12,21,24,44,45|7,24,44|3
sqlite> select * from metrics where version=9 and model_id=3 and matched_size >= 3;
18664|9|8|3|4|2025-05-23 00:16:19.378419|17,9,42,39,21,24|9,24,42|3
18914|9|18|3|4|2025-05-23 00:16:24.277947|5,7,9,15,24,42|7,9,24,42|4
18937|9|19|3|2|2025-05-23 00:16:24.811464|3,5,9,30,40,44|9,40,44|3
18940|9|19|3|5|2025-05-23 00:16:24.811464|3,7,10,18,40,44|7,40,44|3
18961|9|20|3|1|2025-05-23 00:16:25.380846|1,7,14,23,24,40|7,24,40|3
19011|9|22|3|1|2025-05-23 00:16:26.354872|9,24,28,34,42,45|9,24,42|3
19164|9|28|3|4|2025-05-23 00:16:29.209048|1,7,9,20,22,42|7,9,42|3
sqlite> select * from metrics where version=8 and model_id=3 and matched_size >= 3;
17436|8|0|3|1|2025-05-23 00:01:29.843672|8,9,10,24,40,42|9,24,40,42|4
17715|8|11|3|5|2025-05-23 00:01:38.193551|42,22,9,44,6,31|9,42,44|3
18414|8|39|3|4|2025-05-23 00:01:49.318739|5,9,13,18,40,42|9,40,42|3
sqlite> select * from metrics where version=7 and model_id=3 and matched_size >= 3;
16436|7|1|3|1|2025-05-22 23:55:37.709205|42,33,27,17,9,44|9,42,44|3
16614|7|8|3|4|2025-05-22 23:55:40.462202|1,7,9,21,37,44|7,9,44|3
16713|7|12|3|3|2025-05-22 23:55:42.081760|5,36,44,7,24,4|7,24,44|3
16989|7|23|3|4|2025-05-22 23:55:46.422761|7,8,12,28,42,44|7,42,44|3
17088|7|27|3|3|2025-05-22 23:55:47.998755|8,9,21,35,40,44|9,40,44|3
17261|7|34|3|1|2025-05-22 23:55:50.897420|40,24,16,42,4,1|24,40,42|3
17312|7|36|3|2|2025-05-22 23:55:51.696849|1,21,24,35,40,42|24,40,42|3

sqlite> select * from metrics where version=18 and model_id=2 and matched_size >= 3;
27709|18|1|2|4|2025-05-23 17:59:20.027728|44,24,9,23,4,39|9,24,44|3
27910|18|9|2|5|2025-05-23 17:59:24.806816|6,9,22,40,42,43|9,40,42|3
27985|18|12|2|5|2025-05-23 17:59:26.599434|5,11,21,24,40,42|24,40,42|3
28033|18|14|2|3|2025-05-23 17:59:27.816821|1,17,24,40,43,44|24,40,44|3
28159|18|19|2|4|2025-05-23 17:59:30.816944|1,27,30,40,42,44|40,42,44|3
28185|18|20|2|5|2025-05-23 17:59:31.416730|32,28,1,42,24,7|7,24,42|3
28485|18|32|2|5|2025-05-23 17:59:40.422316|22,24,30,40,42,43|24,40,42|3
28506|18|33|2|1|2025-05-23 17:59:41.065684|7,11,15,24,37,40|7,24,40|3
28532|18|34|2|2|2025-05-23 17:59:41.667658|16,39,40,42,43,44|40,42,44|3
28534|18|34|2|4|2025-05-23 17:59:41.667658|6,7,9,18,39,44|7,9,44|3
28607|18|37|2|2|2025-05-23 17:59:43.548926|10,24,33,40,43,44|24,40,44|3
sqlite> select * from metrics where version=17 and model_id=2 and matched_size >= 3;
26881|17|9|2|1|2025-05-23 17:49:24.360264|42,37,44,34,40,45|40,42,44|3
27056|17|16|2|1|2025-05-23 17:49:28.549364|9,16,22,24,33,44|9,24,44|3
27281|17|25|2|1|2025-05-23 17:49:33.979821|10,24,35,40,44,45|24,40,44|3
sqlite> select * from metrics where version=16 and model_id=2 and matched_size >= 3;
25956|16|13|2|1|2025-05-23 17:41:40.381148|12,22,24,27,40,42|24,40,42|3
26057|16|17|2|2|2025-05-23 17:41:42.779068|7,24,20,37,9,3|7,9,24|3
26635|16|40|2|5|2025-05-23 17:41:56.662070|9,15,23,24,32,42|9,24,42|3
sqlite> select * from metrics where version=15 and model_id=2 and matched_size >= 3;
25006|15|16|2|1|2025-05-23 17:33:23.525199|9,22,40,30,43,7|7,9,40|3
25331|15|29|2|1|2025-05-23 17:33:31.501705|9,11,23,24,25,44|9,24,44|3
25382|15|31|2|2|2025-05-23 17:33:32.741582|10,24,26,40,42,45|24,40,42|3
sqlite> select * from metrics where version=14 and model_id=2 and matched_size >= 3;
23835|14|10|2|5|2025-05-23 17:07:35.536111|3,7,13,16,24,42|7,24,42|3
23856|14|11|2|1|2025-05-23 17:07:36.141193|7,8,13,24,41,42|7,24,42|3
24007|14|17|2|2|2025-05-23 17:07:39.736216|40,16,24,27,13,44|24,40,44|3
24059|14|19|2|4|2025-05-23 17:07:41.777075|9,11,24,31,38,44|9,24,44|3
24107|14|21|2|2|2025-05-23 17:07:42.967531|8,9,17,24,32,44|9,24,44|3
24259|14|27|2|4|2025-05-23 17:07:46.558165|7,9,14,16,22,42|7,9,42|3
24458|14|35|2|3|2025-05-23 17:07:51.389247|9,16,20,40,42,44|9,40,42,44|4
sqlite> select * from metrics where version=13 and model_id=2 and matched_size >= 3;
22585|13|1|2|5|2025-05-23 16:58:26.063728|3,9,32,33,42,44|9,42,44|3
22706|13|6|2|1|2025-05-23 16:58:29.047154|5,7,9,28,37,44|7,9,44|3
22709|13|6|2|4|2025-05-23 16:58:29.047154|35,42,7,44,1,37|7,42,44|3
23008|13|18|2|3|2025-05-23 16:58:36.288744|12,34,40,42,43,44|40,42,44|3
23134|13|23|2|4|2025-05-23 16:58:39.312678|8,44,19,22,9,40|9,40,44|3
23182|13|25|2|2|2025-05-23 16:58:40.544113|22,35,39,40,42,44|40,42,44|3
23233|13|27|2|3|2025-05-23 16:58:41.751352|5,6,9,29,40,44|9,40,44|3
23281|13|29|2|1|2025-05-23 16:58:42.954589|1,5,9,41,42,44|9,42,44|3
sqlite> select * from metrics where version=12 and model_id=2 and matched_size >= 3;
21559|12|1|2|4|2025-05-23 16:47:25.777209|5,7,15,24,26,40|7,24,40|3
21881|12|14|2|1|2025-05-23 16:47:35.409900|7,12,18,24,36,42|7,24,42|3
22259|12|29|2|4|2025-05-23 16:47:45.381748|14,25,31,9,7,44|7,9,44|3
22407|12|35|2|2|2025-05-23 16:47:49.367106|6,7,9,38,39,40|7,9,40|3
22507|12|39|2|2|2025-05-23 16:47:52.075608|9,31,8,40,39,42|9,40,42|3
22509|12|39|2|4|2025-05-23 16:47:52.075608|2,9,10,17,24,44|9,24,44|3
sqlite> select * from metrics where version=11 and model_id=2 and matched_size >= 3;
20506|11|0|2|1|2025-05-23 16:32:39.629680|3,4,24,38,42,44|24,42,44|3
20560|11|2|2|5|2025-05-23 16:32:46.286404|7,14,19,40,41,42|7,40,42|3
20807|11|12|2|2|2025-05-23 16:32:53.993242|7,20,24,29,37,42|7,24,42|3
21183|11|27|2|3|2025-05-23 16:33:03.900942|33,1,24,42,7,32|7,24,42|3
21256|11|30|2|1|2025-05-23 16:33:05.799904|7,26,39,40,42,43|7,40,42|3
21385|11|35|2|5|2025-05-23 16:33:08.960052|3,9,24,30,34,42|9,24,42|3
sqlite> select * from metrics where version=10 and model_id=2 and matched_size >= 3;
19485|10|0|2|5|2025-05-23 00:21:10.949664|7,9,13,18,35,42|7,9,42|3
19585|10|4|2|5|2025-05-23 00:21:16.625115|20,24,31,36,40,44|24,40,44|3
19909|10|17|2|4|2025-05-23 00:21:22.097084|10,7,44,18,34,9|7,9,44|3
20110|10|25|2|5|2025-05-23 00:21:25.322401|12,23,37,40,42,44|40,42,44|3
20358|10|35|2|3|2025-05-23 00:21:29.493544|19,7,37,1,40,24|7,24,40|3
sqlite> select * from metrics where version=9 and model_id=2 and matched_size >= 3;
18610|9|6|2|5|2025-05-23 00:16:18.416472|44,42,31,7,14,19|7,42,44|3
18956|9|20|2|1|2025-05-23 00:16:25.380846|29,24,7,5,42,3|7,24,42|3
19231|9|31|2|1|2025-05-23 00:16:30.782262|22,10,40,9,7,21|7,9,40|3
19285|9|33|2|5|2025-05-23 00:16:31.814031|7,12,27,28,42,44|7,42,44|3
19384|9|37|2|4|2025-05-23 00:16:33.869075|5,20,24,31,40,42|24,40,42|3
sqlite> select * from metrics where version=8 and model_id=2 and matched_size >= 3;
17685|8|10|2|5|2025-05-23 00:01:37.808259|36,42,11,40,27,9|9,40,42|3
18108|8|27|2|3|2025-05-23 00:01:44.581348|7,12,17,24,40,41|7,24,40|3
18133|8|28|2|3|2025-05-23 00:01:44.942742|4,7,19,32,40,44|7,40,44|3
18160|8|29|2|5|2025-05-23 00:01:45.379008|7,9,12,22,24,33|7,9,24|3
sqlite> select * from metrics where version=7 and model_id=2 and matched_size >= 3;
16708|7|12|2|3|2025-05-22 23:55:42.081760|24,3,20,38,42,44|24,42,44|3
17032|7|25|2|2|2025-05-22 23:55:47.202197|42,30,22,9,25,24|9,24,42|3
17332|7|37|2|2|2025-05-22 23:55:52.077547|7,9,16,33,34,42|7,9,42|3

sqlite> select * from metrics where version=18 and model_id=1 and matched_size >= 3;
27879|18|8|1|4|2025-05-23 17:59:24.200288|9,15,24,35,39,40|9,24,40|3
28153|18|19|1|3|2025-05-23 17:59:30.816944|7,44,27,40,19,16|7,40,44|3
sqlite> select * from metrics where version=17 and model_id=1 and matched_size >= 3;
26655|17|0|1|5|2025-05-23 17:49:13.127919|3,15,19,24,40,42|24,40,42|3
26827|17|7|1|2|2025-05-23 17:49:23.152138|36,7,42,9,28,1|7,9,42|3
26880|17|9|1|5|2025-05-23 17:49:24.360264|7,9,34,35,42,43|7,9,42|3
27176|17|21|1|1|2025-05-23 17:49:31.568238|3,7,33,40,42,43|7,40,42|3
27254|17|24|1|4|2025-05-23 17:49:33.372352|24,7,44,4,10,29|7,24,44|3
27327|17|27|1|2|2025-05-23 17:49:35.175580|24,42,7,29,20,8|7,24,42|3
27429|17|31|1|4|2025-05-23 17:49:37.633339|24,25,30,35,40,42|24,40,42|3
sqlite> select * from metrics where version=16 and model_id=1 and matched_size >= 3;
25751|16|5|1|1|2025-05-23 17:41:35.631233|7,27,34,40,41,44|7,40,44|3
sqlite> select * from metrics where version=15 and model_id=1 and matched_size >= 3;
24651|15|2|1|1|2025-05-23 17:33:13.756797|5,7,16,24,32,44|7,24,44|3
24952|15|14|1|2|2025-05-23 17:33:22.339750|41,40,7,14,35,44|7,40,44|3
25377|15|31|1|2|2025-05-23 17:33:32.741582|7,9,33,38,39,40|7,9,40|3
25403|15|32|1|3|2025-05-23 17:33:33.341491|6,7,24,25,34,42|7,24,42|3
sqlite> select * from metrics where version=14 and model_id=1 and matched_size >= 3;
23576|14|0|1|1|2025-05-23 17:07:23.786638|24,31,32,39,42,44|24,42,44|3
23578|14|0|1|3|2025-05-23 17:07:23.786638|6,7,16,32,42,44|7,42,44|3
23701|14|5|1|1|2025-05-23 17:07:32.552028|7,9,20,31,32,42|7,9,42|3
24178|14|24|1|3|2025-05-23 17:07:44.768918|8,23,24,41,42,44|24,42,44|3
24329|14|30|1|4|2025-05-23 17:07:48.371960|6,7,9,22,24,28|7,9,24|3
24455|14|35|1|5|2025-05-23 17:07:51.389247|10,24,30,38,42,44|24,42,44|3
24580|14|40|1|5|2025-05-23 17:07:54.416355|35,7,3,21,44,40|7,40,44|3
sqlite> select * from metrics where version=13 and model_id=1 and matched_size >= 3;
23151|13|24|1|1|2025-05-23 16:58:39.908675|4,7,38,40,41,44|7,40,44|3
23179|13|25|1|4|2025-05-23 16:58:40.544113|9,24,37,38,42,45|9,24,42|3
23330|13|31|1|5|2025-05-23 16:58:44.177615|7,9,10,18,43,44|7,9,44|3
sqlite> select * from metrics where version=12 and model_id=1 and matched_size >= 3;
21630|12|4|1|5|2025-05-23 16:47:28.736311|17,19,44,32,24,7|7,24,44|3
21754|12|9|1|4|2025-05-23 16:47:32.070961|44,6,40,28,38,9|9,40,44|3
21954|12|17|1|4|2025-05-23 16:47:37.407734|6,9,24,30,43,44|9,24,44|3
22054|12|21|1|4|2025-05-23 16:47:40.067889|4,9,17,24,42,43|9,24,42|3
22402|12|35|1|2|2025-05-23 16:47:49.367106|26,9,24,8,17,44|9,24,44|3
sqlite> select * from metrics where version=11 and model_id=1 and matched_size >= 3;
20528|11|1|1|3|2025-05-23 16:32:45.701178|5,16,23,24,42,44|24,42,44|3
20777|11|11|1|2|2025-05-23 16:32:51.660969|2,9,13,24,36,42|9,24,42|3
20803|11|12|1|3|2025-05-23 16:32:53.993242|9,11,33,35,40,42|9,40,42|3
20951|11|18|1|1|2025-05-23 16:32:57.611828|7,36,2,3,40,24|7,24,40|3
21229|11|29|1|4|2025-05-23 16:33:05.209697|9,17,19,32,40,42|9,40,42|3
sqlite> select * from metrics where version=10 and model_id=1 and matched_size >= 3;
19603|10|5|1|3|2025-05-23 00:21:16.998539|3,9,10,24,36,44|9,24,44|3
19754|10|11|1|4|2025-05-23 00:21:19.602957|10,7,44,42,21,3|7,42,44|3
20203|10|29|1|3|2025-05-23 00:21:26.970589|9,16,18,24,31,40|9,24,40|3
20278|10|32|1|3|2025-05-23 00:21:28.254032|32,19,44,9,1,24|9,24,44|3
20329|10|34|1|4|2025-05-23 00:21:29.087991|2,3,9,30,40,44|9,40,44|3
20402|10|37|1|2|2025-05-23 00:21:30.458059|17,40,30,9,34,44|9,40,44|3
sqlite> select * from metrics where version=9 and model_id=1 and matched_size >= 3;
18580|9|5|1|5|2025-05-23 00:16:17.958007|7,9,20,24,25,41|7,9,24|3
19029|9|23|1|4|2025-05-23 00:16:26.831484|24,8,37,42,9,1|9,24,42|3
sqlite> select * from metrics where version=8 and model_id=1 and matched_size >= 3;
17451|8|1|1|1|2025-05-23 00:01:34.163053|6,7,31,38,40,44|7,40,44|3
17679|8|10|1|4|2025-05-23 00:01:37.808259|33,44,26,20,42,40|40,42,44|3
17780|8|14|1|5|2025-05-23 00:01:39.378798|18,42,44,9,36,22|9,42,44|3
18279|8|34|1|4|2025-05-23 00:01:47.380730|7,21,24,28,32,44|7,24,44|3
sqlite> select * from metrics where version=7 and model_id=1 and matched_size >= 3;
